In [1]:

import plotly.express as px
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from t_spread_curve import SpreadCurve

In [2]:
tickers = ['MSFT', 'AAPL', 'GOOG', 'AMZN']
features = ['log_tmt', 'tmt', 'coupon']
spread_curve = SpreadCurve(tickers, features, 't_spread')

In [ ]:
spread_curve.t_spread_plots()

In [ ]:
spread_curve.t_spread_plots(dt.datetime(2022, 9, 30))


In [5]:
spread_curve.fit('amount_outstanding')

In [ ]:
spread_curve.training_data.groupby(['date', 'company_symbol'])[['coupon']].std().reset_index().groupby('company_symbol')['coupon'].mean()

In [7]:
query_date = dt.datetime(2021, 1, 31)
subset_train = spread_curve.training_data.query('date == @query_date and company_symbol =="AAPL" ')

In [ ]:
spread_curve.training_data.ncoups.unique()

In [ ]:
for date in spread_curve.bond_data.date.unique():
    spread_curve.plot_fitted_vs_actual(date)

In [ ]:
spread_curve.fitted_models[dt.datetime(2021, 1, 31)]['AAPL']

In [ ]:
spread_curve.training_data['tmt'].round()#[['tmt', 'nearst_pillar']]